# Trabajo Computacional TC_02

##### Heli Magali García Álvarez
##### Santiago Scheiner
##### Juan Ignacio Gossn

# Objetivo principal
## El objetivo es reproducir algunos de los análisis reportados en los trabajos de Zotenko et. al (2008) y He et al. (2006) para las cuatro redes de proteínas de levaduras de las cuales disponemos: Y2H, AP-MS, LIT y LIT-REGULY. La red LIT-REGULY, al igual que LIT es una red construida a partir de interacciones reportadas en la literatura. El trabajo a presentar no sólo deberá mostrar los resultados de estos análisis (tablas y gráficos), sino que también deberá ponerlos en su debido contexto y debatir sobre la naturaleza de los mismos, respetando la estructura usual de un artículo o informe (con, a lo sumo, 10 carillas). A continuación se propone un POSIBLE esquema para el informe:

## A) Introducción
#### Breve introducción sobre la relación entre centralidad y letalidad en redes de proteínas.

## B) Características de las redes analizadas
#### Se deben incluir los siguientes resultados:
#### Tabla 1 de Zotenko et. al. (2008)
#### Tabla 2 de Zotenko et. al. (2008)
#### Figura 1.a de Zotenko et. al. (2008)

## C) Análisis de vulnerabilidad
#### Se deben incluir los siguientes resultados:
#### Figura 3 de Zotenko et. al. (2008)
#### Tabla 3 de Zotenko et. al. (2008)

## D) Esencialidad: Módulos biológicos vs. Interacciónes Esenciales
#### Se deben incluir los siguientes resultados:
#### Figura 2.b de He et. al. (2006) - Para cada una de las redes obtener α y β.
#### Tabla 5 de Zotenko et. al. (2008) - En Número esperado solo deben incluir el obtenido a partir del ajuste lineal.
#### El esquema del informe puede modificarse como ustedes gusten, y pueden agregar cualquier tipo de análisis extra que consideren interesante, pero siempre manteniendo como mínimo los resultados aquí solicitados.

### Bloque de inicialización


Importamos todos los modulos que necesitaremos para el trabajo:


In [2]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp
import itertools

'''
from scipy import stats
from matplotlib import pylab
from sklearn.linear_model import LinearRegression
import random
import matplotlib.patches as mpatches
from statsmodels.stats.proportion import proportions_ztest
import collections
import igraph

from matplotlib_venn import venn3, venn3_circles
'''

'\nfrom scipy import stats\nfrom matplotlib import pylab\nfrom sklearn.linear_model import LinearRegression\nimport random\nimport matplotlib.patches as mpatches\nfrom statsmodels.stats.proportion import proportions_ztest\nimport collections\nimport igraph\n\nfrom matplotlib_venn import venn3, venn3_circles\n'

### Selección de path según la máquina de trabajo:

In [3]:
pathHeli = '/home/heli/Documents/Redes/Practicas/TC_02/'
pathJuancho = '/home/gossn/Dropbox/Documents/Materias_doctorado/RedesComplejasBiologicos/tc02Data/'
pathSanti = '/home/santiago/Documentos/RC/tc02Data/'
pathDocente = '?'

path = pathHeli

Configuraciones para los gráficos:

In [4]:
plt.close('all')
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

TitleSize=20
AxisLabelSize=20
LegendSize=11
NumberSize=12
LabelSize=8 # etiquetas de los nodos
NodeSize=50 # tamaño de los nodos

Definición de función de lectura de las tablas:

In [4]:
'''
def ldata(archivo):

    f=open(archivo)
    data=[]
    for line in f:
        line=line.strip()
        col=line.split()
        data.append(col)
    return data
'''

'\ndef ldata(archivo):\n\n    f=open(archivo)\n    data=[]\n    for line in f:\n        line=line.strip()\n        col=line.split()\n        data.append(col)\n    return data\n'

## Introducción sobre la relación entre centralidad y letalidad en redes de proteínas

En el análisis de grafos, el concepto de *centralidad* consiste en la determinar qué nodos resultan más importantes para la red. Existen diferentes maneras de establecer qué nodos son importantes en una red, y la utilización de cada una dependerá del problema que se esté modelando. [1]

La manera más simple de establecer esta magnitud es la **centralidad de grado**, que consiste en determinar la centralidad en la red a partir del grado de los nodos. En las redes dirigidas puede incluso ser separada en una medida de entrada (*in-degree*) y una de salida (*out-degree*), dos magnitudes que pueden resultar útiles en las circunstancias apropiadas.

Una extensión natural de la centralidad de grado es la **centralidad de autovalor**. En este caso, la importancia de los nodos está dada por sus vecinos, pero teniendo en cuenta a su vez la importancia de cada uno de estos. De este modo, no todos los vecinos de un nodo son equivalentes, y la importancia de un nodo aumenta si tiene conexiones con nodos que son importantes en sí mismos. Por lo tanto, la centralidad de autovalor proporciona a un nodo una importancia que equivale a la suma de las importancias de sus vecinos.La idea es entonces hacer una estimación inicial para la centralidad $x_{i}$ de cada nodo $i$, como puede ser, por ejemplo: $x_{i}=1$. A partir de estos valores iniciales es posible entonces realizar estimaciones de manera iterativa de modo de asignarle a la centralidad $x_{i}$, la suma de las centralidades de sus vecinos del nodo $i$:

$x' = \sum_{j} A_{ij} x_{j}$

donde $A_{ij}$ es un elemento de la matriz de adyacencia. Escrito de manera matricial, se tiene: $x' = Ax$, lo que significa que $x$ converge a un autovalor de la matriz de adyacencia.

La centralidad de autovalor puede ser calculada tanto para redes dirigidas como para no dirigidas. Sin embargo, para el caso de las redes dirigidas, existe la posibilidad de que el método detallado antes no funcione correctamente. Uno de los inconvenientes que surgen con esta definición de centralidad para redes dirigidas ocurre cuando un nodo únicamente se conecta con otros mediante enlaces salientes. Este vértice tendrá entonces centralidad nula, lo que puede propagarse a los nodos apuntados por el primero, si estos otros solo son apuntados por nodos con centralidad nula. Este fenómeno puede ocurrir entonces nuevamente y propagarse por la red, lo que dará como resultado una centralidad nula para todos los nodos.

Una solución posible a los problemas que ocurren con la definición anterior es utilizar la **centralidad de Katz**. La manera de calcular esta magnitud es igual a la utilizada para la centralidad de autovalor, pero agregando un término pequeño $\beta$. Puede pensarse a $\beta$ como la asignación de una centralidad de manera *gratuita* (en el sentido de que este término no depende de la posición del nodo en la red) a la centralidad de autovalor:

$x_{i} = \alpha \sum A_{ij} \; x_{j} + \beta$

donde $\alpha$ y $\beta$ son constantes positivas. Puede verse entonces que esto previene el inconveniente que se daba para la centralidad de autovalor para gráficos dirigidos, ya que en este caso ya no se tienen nodos con centralidad nula.

La centralidad de Katz también tiene una característica que puede resultar indeseada en algunos casos. Si un nodo con un alto grado de centralidad apunta a muchos otros nodos, todos estos reciben también un alto índice de centralidad, lo que en muchos casos no resulta apropiado (como es el caso de los motores de búsqueda). Una buena manera de evitar este problema es entonces mediante el uso de la **centralidad PageRank**, que consiste en una adaptación de la centralidad de Katz definida de la siguiente manera:

$x_{i} = \alpha \sum A_{ij} \dfrac{x_{j}}{k_{j}^{out}} + \beta $

De esta forma, la centralidad de un nodo con un alto grado de salida $k^{out}$ se ve *diluida* entre todos los nodos a los que apunta. En otras palabras, el hecho de que un nodo sea apuntado junto a muchos otros le quita parte de esta *importancia heredada*.

Relacionada con las últimas definiciones de centralidad, en que la importancia de un nodo está determinada por la importancia de los nodos que lo apuntan, aparecen los conceptos de **hub y autoridades**. Para algunas redes, puede ser importante determinar qué nodos apuntan a nodos relevantes (como por ejemplo, en el caso de redes de citas de artículos, en el que un artículo puede no contener demasiada información sobre un tema dado, pero puede indicar qué otros artículos contienen información valiosa sobre ese tema). En este sentido, se denominan **autoridades** a los nodos relevantes (artículos que contienen información útil, por ejemplo) y **hubs** a los nodos que apuntan a las autoridades.

La centralidad de autoridad $x_{i}$ de un nodo puede ser definida de manera proporcional a la suma de las centralidades de *hub* $y_{j}$ de los nodos que lo apuntan:

$x_{i} = \alpha \sum A_{ij} y_{j}$

Asimismo, las centralidad de *hub* $y_{i}$ de un nodo puede definirse de manera proporcional a la suma de las centralidades de autoridad de los nodos a los que apunta: 

$y_{i} = \beta \sum A_{ji} x_{j}$

Una manera completamente diferente de medir la centralidad de los nodos es la **centralidad de cercanía**, que mide la distancia que lo separa de los otros nodos de la red. Si $d_{ij}$ es la distancia mínima desde el nodo $i$ hasta el nodo $j$, la media de las distancias mínimas desde el nodo $i$ hasta los demás nodos es:

$l_{i} = \dfrac{1}{n} \sum d_{ij}$

Entonces, la centralidad de cercanía puede calcularse como:

$C_{i} = \dfrac{1}{l_{i}}$

Otra manera de medir la centralidad, que tampoco está relacionada con las anteriores es la **centralidad de intermediación** de un nodo, que consiste establecer cuántos caminos mínimos entre otros dos nodos pasan por el nodo que se analiza. En este sentido, un nodo se vuelve relevante cuando comunica muchos otros nodos entre sí, tomando el camino óptimo entre ellos.

Una variante de esta última definición es la **centralidad de intermediación de caminos al azar**, que en lugar de tener en cuenta únicamente los caminos mínimos, tiene en cuenta todos los caminos tomados de manera aleatoria que unan a ambos nodos.

### Relación entre centralidad y letalidad:

##### Abstract del paper de Zotenko:

The centrality-lethality rule, which notes that high-degree nodes in a protein interaction network tend to correspond to
proteins that are essential, suggests that the topological prominence of a protein in a protein interaction network may be a
good predictor of its biological importance. Even though the correlation between degree and essentiality was confirmed by
many independent studies, the reason for this correlation remains illusive. Several hypotheses about putative connections
between essentiality of hubs and the topology of protein–protein interaction networks have been proposed, but as we
demonstrate, these explanations are not supported by the properties of protein interaction networks. To identify the main
topological determinant of essentiality and to provide a biological explanation for the connection between the network
topology and essentiality, we performed a rigorous analysis of six variants of the genomewide protein interaction network
for *Saccharomyces cerevisiae* obtained using different techniques. We demonstrated that the majority of hubs are essential
due to their involvement in Essential Complex Biological Modules, a group of densely connected proteins with shared
biological function that are enriched in essential proteins. Moreover, we rejected two previously proposed explanations for
the centrality-lethality rule, one relating the essentiality of hubs to their role in the overall network connectivity and another
relying on the recently published essential protein interactions model.




#### Referencias:

[1] Newman, M.E.J. 2010. Networks: An Introduction. Oxford, UK: Oxford University Press.

In [5]:
redesStr = ['Y2H','AP-MS','LIT','LIT_Reguly']
redes = {}

for s in redesStr:
    df = pd.read_csv(path + 'yeast_' + s + '.txt', sep = '\t')
    if s == 'LIT_Reguly':
        net = df[['Bait gene/protein','Hit gene/protein']].values.tolist()
    else:
        net = df.values.tolist()
    redes[s] = nx.Graph(net)

df = pd.read_csv(path + 'Essential_ORFs_paperHe.txt', sep='\t')
esenciales = df['ORF_name'].values.tolist()

#borramos el primer y los dos ultimos elementos de la lista esenciales
del esenciales[0]
del esenciales[len(esenciales)-1]
del esenciales[len(esenciales)-1]

print(esenciales)

#sacamos los espacios al final de cada nombre de la lista
essentials = []
for item in range(len(esenciales)):
    essentials.append(esenciales[item].strip())
    
print(essentials)

['YAL001C  ', 'YAL003W  ', 'YAL025C  ', 'YAL032C  ', 'YAL033W  ', 'YAL034W-a', 'YAL035C-A', 'YAL038W  ', 'YAL041W  ', 'YAL043C  ', 'YAR007C  ', 'YAR008W  ', 'YAR019C  ', 'YBL004W  ', 'YBL014C  ', 'YBL018C  ', 'YBL020W  ', 'YBL023C  ', 'YBL026W  ', 'YBL030C  ', 'YBL034C  ', 'YBL035C  ', 'YBL040C  ', 'YBL041W  ', 'YBL050W  ', 'YBL073W  ', 'YBL074C  ', 'YBL076C  ', 'YBL077W  ', 'YBL084C  ', 'YBL092W  ', 'YBL097W  ', 'YBL105C  ', 'YBR002C  ', 'YBR004C  ', 'YBR011C  ', 'YBR029C  ', 'YBR038W  ', 'YBR049C  ', 'YBR055C  ', 'YBR060C  ', 'YBR070C  ', 'YBR079C  ', 'YBR080C  ', 'YBR087W  ', 'YBR087W  ', 'YBR088C  ', 'YBR089W  ', 'YBR091C  ', 'YBR102C  ', 'YBR109C  ', 'YBR110W  ', 'YBR123C  ', 'YBR124W  ', 'YBR135W  ', 'YBR136W  ', 'YBR140C  ', 'YBR142W  ', 'YBR143C  ', 'YBR152W  ', 'YBR153W  ', 'YBR154C  ', 'YBR155W  ', 'YBR160W  ', 'YBR167C  ', 'YBR190W  ', 'YBR192W  ', 'YBR193C  ', 'YBR196C  ', 'YBR198C  ', 'YBR202W  ', 'YBR211C  ', 'YBR233W-A', 'YBR234C  ', 'YBR236C  ', 'YBR237W  ', 'YBR243C  '

In [ ]:
'''Tabla 1 de Zotenko et. al. (2008)'''

dfB1 = pd.DataFrame()

for s in redesStr:
	dfB1.loc[s,'Nodes'] = redes[s].number_of_nodes()
	dfB1.loc[s,'Edges'] = redes[s].number_of_edges()
	if len(redes[s]) == len(np.unique(redes[s],axis=0)):
		dfB1.loc[s,'Directionality'] = 'Prob-Undir'
	else:
		dfB1.loc[s,'Directionality'] = 'Dir'
	netDeg = np.array(list(redes[s].degree()))
	netDeg = netDeg[:,1]
	netDeg = netDeg.astype(int)
	dfB1.loc[s,'DegMean'] = np.mean(netDeg)
	dfB1.loc[s,'DegMin'] = np.min(netDeg)
	dfB1.loc[s,'DegMax'] = np.max(netDeg)
	dfB1.loc[s,'DegDensity'] = nx.density(redes[s])
	dfB1.loc[s,'C_tri'] = nx.transitivity(redes[s])
	dfB1.loc[s,'C_avg'] = nx.average_clustering(redes[s])
	giant = max(nx.connected_component_subgraphs(redes[s]), key=len)
	dfB1.loc[s,'Diameter (Giant Subgraph)'] = nx.diameter(giant)

print(dfB1)

In [8]:
'''Tabla 2 de Zotenko et. al. (2008)'''

#primero chequeo que las redes no tengan enlaces repetidos

for s in redesStr:
    if len(list(redes[s].edges()))==len(np.unique(list(redes[s].edges()), axis=0)):
        print("No hay enlaces repetidos para la red " + s)
    else:
        print("Hay enlaces repetidos para la red " + s)

#calculo la cantidad de enlaces de una red que estan tambien presentes en las restantes

iter=list(itertools.combinations(range(len(redesStr)),2))

def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2)) 

ratios = np.zeros((4, 4), float)
np.fill_diagonal(ratios, 1)

for (j,k) in iter:
    red_j = redes[redesStr[j]].copy()
    red_k = redes[redesStr[k]].copy()
    
    commonNodes = intersection(list(red_j), list(red_k))
    red_j.remove_nodes_from(list(nodoi for nodoi in red_j if nodoi not in commonNodes))
    red_k.remove_nodes_from(list(nodoi for nodoi in red_k if nodoi not in commonNodes))
    
    red_jk = nx.intersection(red_j,red_k)
    commonEdges = red_jk.number_of_edges()
    
    ratios[j,k] = commonEdges/(redes[redesStr[j]].number_of_edges())
    ratios[k,j] = commonEdges/(redes[redesStr[k]].number_of_edges())

dfB2 = pd.DataFrame(ratios, columns=redesStr)
dfB2.index = redesStr

print(dfB2)

No hay enlaces repetidos para la red Y2H
No hay enlaces repetidos para la red AP-MS
No hay enlaces repetidos para la red LIT
No hay enlaces repetidos para la red LIT_Reguly
                 Y2H     AP-MS       LIT  LIT_Reguly
Y2H         1.000000  0.088767  0.088426    0.163537
AP-MS       0.028669  1.000000  0.142904    0.277759
LIT         0.088577  0.443228  1.000000    0.977770
LIT_Reguly  0.040395  0.212430  0.241103    1.000000


In [ ]:
'''Figura 1.a de Zotenko et. al. (2008)'''

percentiles = np.linspace(0,100,num=101)

degree_sequence = {}
degree_cutoff = {}
ratio_hubs_essentials = {}

for k in redesStr:
    degree_sequence[k] = sorted([d for n, d in redes[k].degree()])
    deg_cutoff = []
    for i in range(len(percentiles)):
        deg_cutoff.append(np.percentile(degree_sequence[k], i))
    degree_cutoff[k] = deg_cutoff

    ratio = []
    for j in range(len(deg_cutoff)):
        count_hubs = 0
        count_hubs_essentials = 0
        for nodoi in list(redes[k].nodes()):
            if redes[k].degree(nodoi)>=deg_cutoff[j]:
                count_hubs+=1
                if nodoi in essentials:                                                                            
                    count_hubs_essentials+=1
        ratio.append(count_hubs_essentials/count_hubs)
    ratio_hubs_essentials[k]= ratio


x = []
for i in range(len(percentiles)):
    x.append(1 - percentiles[i]/100)


plt.figure()
for k in redesStr:
    plt.plot(x, ratio_hubs_essentials[k], linewidth=3, label=k)
plt.xticks(np.arange(0, 1.1, step=0.1))
plt.yticks(np.arange(0, 1.1, step=0.1))
plt.tick_params(axis='both', which='major', labelsize=NumberSize)
plt.xlabel('Hub definition cutoff', fontsize=20)
plt.ylabel('Fraction of essential nodes', fontsize=20)   
plt.title('Figure 1A. Relationship between degree and essentiality in the tested networks', fontsize=30)
plt.legend(loc='upper right', fontsize=20) 
plt.grid(axis='both', color='k', linestyle='dashed', linewidth=2, alpha=0.1)
plt.show()